# File Selection Pagination

> Factory function for creating paginated file selection table with radio buttons

In [ ]:
#| default_exp media.file_selection_pagination

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Any, Optional, Callable
from fasthtml.common import *

from cjm_fasthtml_interactions.patterns.pagination import Pagination
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_styles, btn_sizes
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.data_display.table import table, table_modifiers
from cjm_fasthtml_daisyui.components.data_input.radio import radio
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.utilities.typography import text_align, font_size
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_workflow_transcription_single_file.media.models import MediaFile

from cjm_fasthtml_workflow_transcription_single_file.media.scanner import MediaScanner

In [ ]:
#| export
def _escape_js(
    s: str  # String to escape
) -> str:  # Escaped string safe for JavaScript
    """Escape a string for use in JavaScript."""
    return s.replace("\\", "\\\\").replace("'", "\\'").replace('"', '\\"').replace("\n", "\\n")

In [ ]:
#| export
def _render_file_row(
    file: MediaFile,  # MediaFile to render
    idx: int,  # Global index of this file (across all pages)
    selected_file: Optional[str],  # Currently selected file path (if any)
    preview_url_func: Optional[Callable[[int], str]],  # Function to generate preview URL
    preview_target_id: Optional[str]  # Target ID for preview modal
) -> FT:  # Table row element
    """Render a single file row in the selection table."""
    is_selected = file.path == selected_file

    return Tr(
        Td(
            Input(
                type="radio",
                name="file_path",
                value=file.path,
                checked=is_selected,
                cls=str(radio),
                required=True,
                # Store additional file info as hidden inputs when selected
                onchange=f"""
                    document.getElementById('file_name').value = '{_escape_js(file.name)}';
                    document.getElementById('file_type').value = '{file.media_type}';
                    document.getElementById('file_size').value = '{file.size_str}';
                """
            )
        ),
        Td(
            file.name[:40] + "..." if len(file.name) > 40 else file.name,
            title=file.name
        ),
        Td(
            Span(
                file.media_type.upper(),
                cls=combine_classes(
                    badge,
                    badge_colors.primary if file.media_type == "video" else badge_colors.secondary,
                    badge_sizes.sm
                )
            )
        ),
        Td(file.size_str),
        Td(file.modified_str),
        Td(
            Button(
                "Preview",
                type="button",
                hx_get=preview_url_func(idx) if preview_url_func else None,
                hx_target=f"#{preview_target_id}" if preview_target_id else None,
                hx_swap="innerHTML",
                cls=combine_classes(btn, btn_styles.ghost, btn_sizes.xs)
            )
        ) if preview_url_func else None,
        cls=str(bg_dui.base_200) if is_selected else str(bg_dui.base_200.hover)
    )

## create_file_selection_pagination

Factory function that creates a `Pagination` instance configured for file selection in the transcription workflow. Displays files in a table with radio buttons for selection.

In [ ]:
#| export
def create_file_selection_pagination(
    pagination_id: str,  # Unique identifier for this pagination instance
    scanner: MediaScanner,  # MediaScanner instance for loading files
    items_per_page: int = 30,  # Number of items per page
    content_id: Optional[str] = None,  # HTML ID for content area
    preview_url_func: Optional[Callable[[int], str]] = None,  # Function that takes file index and returns preview URL
    preview_target_id: Optional[str] = None  # HTML ID to target for preview modal
) -> Pagination:  # Configured Pagination instance for file selection
    """Create a Pagination instance for file selection with radio buttons."""

    def load_media_files(request) -> List[MediaFile]:
        """Load transcribable media files from scanner."""
        files = scanner.scan()
        # Filter to only audio/video (transcribable files)
        return [f for f in files if f.media_type in ["audio", "video"]]

    def render_file_selection_table(items: List[MediaFile], page: int, request) -> Any:
        """Render file selection table for the current page."""
        # Get selected file from query params (if any)
        selected_file = request.query_params.get("selected")

        # Calculate the global start index for this page
        start_idx = (page - 1) * items_per_page

        if not items:
            return Div(
                P(
                    "No audio or video files found on this page.",
                    cls=combine_classes(text_dui.base_content.opacity(60), text_align.center, p(8))
                )
            )

        return Table(
            Thead(
                Tr(
                    Th("Select", cls=str(w(12))),
                    Th("Name"),
                    Th("Type", cls=str(w(20))),
                    Th("Size", cls=str(w(24))),
                    Th("Modified", cls=str(w(28))),
                    Th("Preview", cls=str(w(20))) if preview_url_func else None
                )
            ),
            Tbody(
                *[
                    _render_file_row(
                        file=file,
                        idx=start_idx + i,
                        selected_file=selected_file,
                        preview_url_func=preview_url_func,
                        preview_target_id=preview_target_id
                    )
                    for i, file in enumerate(items)
                ]
            ),
            cls=combine_classes(table, table_modifiers.zebra, w.full)
        )

    return Pagination(
        pagination_id=pagination_id,
        data_loader=load_media_files,
        render_items=render_file_selection_table,
        items_per_page=items_per_page,
        content_id=content_id,
        preserve_params=["selected"],
        show_endpoints=True,
        push_url=False  # Don't push URL for workflow-internal browsing
    )

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()